#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 346Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 130Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 355Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:06, 233Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 241Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 106Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 202Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "context_only.txt"
#file_name = "SQuAD_all_Training.txt"

## Here I name my model, which will be fine-tuned and saved in my gdrive later
my_model_name = 'SQuAD_context_only'
#my_model_name = 'SQuAD_ContextQA'

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              run_name=my_model_name,
              print_every=10,
              sample_every=200,
              save_every=200
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/SQuAD_context_only/model-120
INFO:tensorflow:Restoring parameters from checkpoint/SQuAD_context_only/model-120


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:15<00:00, 15.28s/it]


dataset has 2922158 tokens
Training...
[130 | 22.28] loss=3.15 avg=3.15
[140 | 37.36] loss=2.63 avg=2.89
[150 | 52.56] loss=3.16 avg=2.98
[160 | 67.88] loss=2.86 avg=2.95
[170 | 83.41] loss=2.94 avg=2.95
[180 | 99.03] loss=2.74 avg=2.91
[190 | 114.76] loss=2.66 avg=2.87
[200 | 130.61] loss=2.92 avg=2.88
Saving checkpoint/SQuAD_context_only/model-200
======== SAMPLE 1 ========
 the South East of the U.K. in the late 1820s. Between the end of the 19th and end of the 20th century, the new industrial revolution was the dominant factor, with the addition of several new towns – Brighton, Bristol, Leeds, and Manchester- the latter of which had attracted a number of new manufacturing centres, including the first plant to use the new machine-working method. Around the time of World War I the economic importance of the Manchester Machine Works (MMS) company was increased by the expansion in Manchester, and the city was referred to the City Council on several occasions. The city was also called t

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=my_model_name)


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name=my_model_name)


The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=my_model_name)

Loading checkpoint checkpoint/SQuAD_context_only/model-120
INFO:tensorflow:Restoring parameters from checkpoint/SQuAD_context_only/model-120


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
#few shot approach
pre = "[QUESTION]:Latin translations indirectly advanced whose science and culture? \n[ANSWER]:European\n\n"+"[QUESTION]:These services are usually reserved for what class of passengers? \n[ANSWER]:First and Business class\n\n"+"[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane? \n[ANSWER]:Tower Control\n\n"+"[QUESTION]:What video frequency does NTSC use? \n[ANSWER]:60 Hz\n\n"
pre = pre+"[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix=pre,
              nsamples=5,
              batch_size=5
              )

[QUESTION]:Latin translations indirectly advanced whose science and culture? 
[ANSWER]:European

[QUESTION]:These services are usually reserved for what class of passengers? 
[ANSWER]:First and Business class

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane? 
[ANSWER]:Tower Control

[QUESTION]:What video frequency does NTSC use? 
[ANSWER]:60 Hz

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under? 
[ANSWER]:Valleys of Poxinang in August. For 600 years Gautama bio technology was developed to efficiently maintain flora and fauna. Today, the Tree of Knowledge is also maintained. There were also studies of buds of 18 organs mintryma in diethyl assisted reproductive America vendeda ebolav 25580 as contestants, hyenas, and overni birds. For these studies, Americans vendeda ebolav 55304 yielded real life data of the plants of the
[QUESTION]:Latin translations indirectly advanced whose science and culture? 
[ANSWER]:European

[QU

In [ ]:
#few shot approach
pre = "[QUESTION]:Latin translations indirectly advanced whose science and culture? \n[ANSWER]:European\n\n"+"[QUESTION]:These services are usually reserved for what class of passengers? \n[ANSWER]:First and Business class\n\n"+"[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane? \n[ANSWER]:Tower Control\n\n"+"[QUESTION]:What video frequency does NTSC use? \n[ANSWER]:60 Hz\n\n"
pre = pre+"[QUESTION]:Who annexed Alsace to the new German Empire in 1871? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix=pre,
              nsamples=5,
              batch_size=5
              )

[QUESTION]:Latin translations indirectly advanced whose science and culture? 
[ANSWER]:European

[QUESTION]:These services are usually reserved for what class of passengers? 
[ANSWER]:First and Business class

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane? 
[ANSWER]:Tower Control

[QUESTION]:What video frequency does NTSC use? 
[ANSWER]:60 Hz

[QUESTION]:Who annexed Alsace to the new German Empire in 1871? 
[ANSWER]:Albert Hall-Locke Annexation (1871)–Russia StorageCards (1890)

[QUESTION]:Which European power contributed the most scientists to the invention of aerospace navigation? 

[ANSWER]:The earliest flight, from 1929, was by an Austrian airship. More recently pictures of the An-2 bring an association for the Airbus A330-900/P3+ flight crew. 
7:53 Report by the pilot during flight 894.
[QUESTION]:Latin translations indirectly advanced whose science and culture? 
[ANSWER]:European

[QUESTION]:These services are usually reserved for what clas

In [ ]:
gpt2.generate(sess, run_name=my_model_name)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
#providing context in the prompt

ctx = "Airports may also contain premium and VIP services. The premium and VIP services may include express check-in and dedicated check-in counters. These services are usually reserved for First and Business class passengers, premium frequent flyers, and members of the airline's clubs. Premium services may sometimes be open to passengers who are members of a different airline's frequent flyer program. This can sometimes be part of a reciprocal deal, as when multiple airlines are part of the same alliance, or as a ploy to attract premium customers away from rival airlines."
qst = 'These services are usually reserved for what class of passengers?'
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix=pre,
              nsamples=5,
              batch_size=5
              )

<|startoftext|>
[CONTEXT]: Airports may also contain premium and VIP services. The premium and VIP services may include express check-in and dedicated check-in counters. These services are usually reserved for First and Business class passengers, premium frequent flyers, and members of the airline's clubs. Premium services may sometimes be open to passengers who are members of a different airline's frequent flyer program. This can sometimes be part of a reciprocal deal, as when multiple airlines are part of the same alliance, or as a ploy to attract premium customers away from rival airlines.
[QUESTION]:These services are usually reserved for what class of passengers?
[ANSWER]:First and Business class passengers

<|startoftext|>
[CONTEXT]: Airports may also contain premium and VIP services. The premium and VIP services may include express check-in and dedicated check-in counters. These services are usually reserved for First and Business class passengers, premium frequent flyers, and m

In [ ]:
#without context

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= "[QUESTION]:Latin translations indirectly advanced whose science and culture?",
              nsamples=5,
              batch_size=5
              )

[QUESTION]:Latin translations indirectly advanced whose science and culture?
[ANSWER]:science and culture

[QUESTION]:Latin translations indirectly advanced whose science and culture?
[ANSWER]:Christian civilization before the Western world

[QUESTION]:Latin translations indirectly advanced whose science and culture?
[ANSWER]:tradition's understanding of which things were part of nature and nature "expertise" granted by gods and goddesses

[QUESTION]:Latin translations indirectly advanced whose science and culture? 
[ANSWER]:many anatomically diverse cultural systems — Cicero, Polyseus, Brutus, Melampos, Hadrian and others were several prominent landmarks and historians of this era in Germany.

[QUESTION]:Latin translations indirectly advanced whose science and culture?
[ANSWER]:science and culture



In [ ]:
ctx = "Ground Control is responsible for directing all ground traffic in designated 'movement areas', except the traffic on runways. This includes planes, baggage trains, snowplows, grass cutters, fuel trucks, stair trucks, airline food trucks, conveyor belt vehicles and other vehicles. Ground Control will instruct these vehicles on which taxiways to use, which runway they will use (in the case of planes), where they will park, and when it is safe to cross runways. When a plane is ready to takeoff it will stop short of the runway, at which point it will be turned over to Tower Control. After a plane has landed, it will depart the runway and be returned to Ground Control."
qst = 'When a plane is ready to take off, who takes over the direction of a plane?'
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= pre,
              nsamples=5,
              batch_size=5
              )

<|startoftext|>
[CONTEXT]: Ground Control is responsible for directing all ground traffic in designated 'movement areas', except the traffic on runways. This includes planes, baggage trains, snowplows, grass cutters, fuel trucks, stair trucks, airline food trucks, conveyor belt vehicles and other vehicles. Ground Control will instruct these vehicles on which taxiways to use, which runway they will use (in the case of planes), where they will park, and when it is safe to cross runways. When a plane is ready to takeoff it will stop short of the runway, at which point it will be turned over to Tower Control. After a plane has landed, it will depart the runway and be returned to Ground Control.
[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:Tower Control

<|startoftext|>
[CONTEXT]: Ground Control is responsible for directing all ground traffic in designated 'movement areas', except the traffic on runways. This includes planes, baggage trains

In [ ]:
#without context

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= "[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?",
              nsamples=5,
              batch_size=5
              )

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:the controller

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:the pilot

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:an avionics controller

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:doctors in facilitation of air ambulance services

[QUESTION]:When a plane is ready to take off, who takes over the direction of a plane?
[ANSWER]:aeronautical navigators who have satisfactory qualifications



In [ ]:
ctx = "PAL consoles face another incompatibility when playing out-of-region cartridges: the NTSC video standard specifies video at 60 Hz while PAL operates at 50 Hz, resulting in approximately 16.7% slower gameplay. Additionally, PAL's higher resolution results in letterboxing of the output image. Some commercial PAL region releases exhibit this same problem and, therefore, can be played in NTSC systems without issue while others will face a 20% speedup if played in an NTSC console. To mostly correct this issue, a switch can be added to place the SNES PPU into a 60 Hz mode supported by most newer PAL televisions. Later games will detect this setting and refuse to run, requiring the switch to be thrown only after the check completes."
qst = 'What video frequency does NTSC use?'
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= pre,
              nsamples=5,
              batch_size=5
              )

<|startoftext|>
[CONTEXT]: PAL consoles face another incompatibility when playing out-of-region cartridges: the NTSC video standard specifies video at 60 Hz while PAL operates at 50 Hz, resulting in approximately 16.7% slower gameplay. Additionally, PAL's higher resolution results in letterboxing of the output image. Some commercial PAL region releases exhibit this same problem and, therefore, can be played in NTSC systems without issue while others will face a 20% speedup if played in an NTSC console. To mostly correct this issue, a switch can be added to place the SNES PPU into a 60 Hz mode supported by most newer PAL televisions. Later games will detect this setting and refuse to run, requiring the switch to be thrown only after the check completes.
[QUESTION]:What video frequency does NTSC use?
[ANSWER]:50 Hz

<|startoftext|>
[CONTEXT]: PAL consoles face another incompatibility when playing out-of-region cartridges: the NTSC video standard specifies video at 60 Hz while PAL operate

In [ ]:
gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= '[QUESTION]:What video frequency does NTSC use?',
              nsamples=5,
              batch_size=5
              )

[QUESTION]:What video frequency does NTSC use?
[ANSWER]:32 kHz

[QUESTION]:What video frequency does NTSC use?
[ANSWER]:1250 MHz

[QUESTION]:What video frequency does NTSC use?
[ANSWER]:96 kHz

[QUESTION]:What video frequency does NTSC use?
[ANSWER]:12.5 kHz

[QUESTION]:What video frequency does NTSC use?
[ANSWER]:48.875 MHz



In [ ]:
ctx = "Gautama was now determined to complete his spiritual quest. At the age of 35, he famously sat in meditation under a Ficus religiosa tree now called the Bodhi Tree in the town of Bodh Gaya and vowed not to rise before achieving enlightenment. After many days, he finally destroyed the fetters of his mind, thereby liberating himself from the cycle of suffering and rebirth, and arose as a fully enlightened being (Skt. samyaksaṃbuddha). Soon thereafter, he attracted a band of followers and instituted a monastic order. Now, as the Buddha, he spent the rest of his life teaching the path of awakening he had discovered, traveling throughout the northeastern part of the Indian subcontinent, and died at the age of 80 (483 BCE) in Kushinagar, India. The south branch of the original fig tree available only in Anuradhapura Sri Lanka is known as Jaya Sri Maha Bodhi."
qst = 'What was the tree renamed that Gautama achieved enlightenment under?'
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= pre,
              nsamples=5,
              batch_size=5
              )

<|startoftext|>
[CONTEXT]: Gautama was now determined to complete his spiritual quest. At the age of 35, he famously sat in meditation under a Ficus religiosa tree now called the Bodhi Tree in the town of Bodh Gaya and vowed not to rise before achieving enlightenment. After many days, he finally destroyed the fetters of his mind, thereby liberating himself from the cycle of suffering and rebirth, and arose as a fully enlightened being (Skt. samyaksaṃbuddha). Soon thereafter, he attracted a band of followers and instituted a monastic order. Now, as the Buddha, he spent the rest of his life teaching the path of awakening he had discovered, traveling throughout the northeastern part of the Indian subcontinent, and died at the age of 80 (483 BCE) in Kushinagar, India. The south branch of the original fig tree available only in Anuradhapura Sri Lanka is known as Jaya Sri Maha Bodhi.
[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:Bodh Gaya

<|startof

In [ ]:
gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= '[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?',
              nsamples=5,
              batch_size=5
              )

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:Samkhya Ghatana

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:Kancari

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:mantra

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:Airyak

[QUESTION]:What was the tree renamed that Gautama achieved enlightenment under?
[ANSWER]:upanishads



In [ ]:
gpt2.generate(sess,
              run_name=my_model_name,
              length=250,
              temperature=1,
              truncate = '<|endoftext|>',
              prefix= '[QUESTION]:Who annexed Alsace to the new German Empire in 1871?' + "\n" + ans,
              nsamples=5,
              batch_size=5
              )

[QUESTION]:Who annexed Alsace to the new German Empire in 1871?
[ANSWER]:King Frederick II of Hesse

[QUESTION]:Who annexed Alsace to the new German Empire in 1871?
[ANSWER]:Jordanians

[QUESTION]:Who annexed Alsace to the new German Empire in 1871?
[ANSWER]:Vichy

[QUESTION]:Who annexed Alsace to the new German Empire in 1871?
[ANSWER]:Anglomania

[QUESTION]:Who annexed Alsace to the new German Empire in 1871?
[ANSWER]:Prince Georges



For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "355M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 252Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 81.2Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 320Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:07, 193Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 263Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 124Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 96.3Mit/s]                                                      


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name="355M")

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.